![StatModels](https://www.durhamtech.edu/themes/custom/durhamtech/images/durham-tech-logo-web.svg) 

## Applications - Statistical Modeling

This lecture provides foundational knowledge and examples of machine learning modeling concepts by examining stock price data.

---

# Table of Contents

### Jupyter Overview
#### <a href='#1'>Useful Links</a>
#### <a href='#2'>Introduction to Jupyter Notebooks</a>
#### <a href='#3'>Cell Types</a>
* Markdown 
* Code
    1. Running One Cell
    2. Other Run Options

#### <a href='#4'>Tips and Tricks</a>

#### <a href='#55'>Weekly Readings/Videos</a>
#### <a href='#56'>Extra Practice</a>

## Introduction

![FunnyML](https://www.meme-arsenal.com/memes/11f11b5d16eef661677e4c9e989dd2b3.jpg) 

## Data Sources

1. Weather: https://mesonet.agron.iastate.edu/request/download.phtml?network=WI_ASOS
2. Weather python: https://github.com/akrherz/iem/blob/main/scripts/asos/iem_scraper_example.py
3. SP 500 Components: https://datahub.io/core/s-and-p-500-companies
4. SP 500 Company Info: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
5. FRED https://fred.stlouisfed.org/docs/api/fred/series.html
6. TD Ameritrade Data Dicitionary https://developer.tdameritrade.com/content/streaming-data#_Toc504640567



In [ ]:
# https://pypi.org/project/yahoo-finance-api2/
# https://github.com/pkout/yahoo_finance_api2

# Uncomment below if you don't have yahoo finance api installed
# pip install yahoo_finance_api2

In [144]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import math

from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

import warnings
import matplotlib.pyplot as plt
import numpy as np
import sklearn

import json
import datetime

warnings.filterwarnings('ignore')
key = 'RGOLSJPSTGVAN4NTN4DLWJE71SU7SIH0'

In [349]:
# view file contents
ls

20120801.txt                Statistical_Modeling.ipynb
SP500_info.csv              constituents_csv.csv


In [46]:
tickers=pd.read_csv("constituents_csv.csv")
print(len(tickers))
tickers.head()

505


,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [47]:
ticker_info=pd.read_csv("sp500_info.csv")
print(len(ticker_info))
ticker_info.head()

505


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",8/9/76,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/64,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/12,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/18,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/11,1467373,1989


In [49]:
tickers=pd.merge(tickers,ticker_info,on='Symbol',how='inner')
print(len(tickers))
tickers.head()

505


,Symbol,Name,Sector,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,Industrials,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",8/9/76,66740,1902
1,AOS,A. O. Smith,Industrials,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",7/26/17,91142,1916
2,ABT,Abbott Laboratories,Health Care,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/64,1800,1888
3,ABBV,AbbVie,Health Care,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/12,1551152,2013 (1888)
4,ABMD,Abiomed,Health Care,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/18,815094,1981


In [50]:
del ticker_info

In [180]:
def get_td_price_hist(ticker,period,key,row_count='Blank'):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+ticker+'/pricehistory'

    ##Define Payload
    payload = {'apikey': key,
    'periodType': 'year',
    'period':period,
    'frequencyType':'daily'}

    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()

    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        hist_data = pd.DataFrame(dictlist[0][1])
        hist_data['datetime'] = pd.to_datetime(hist_data['datetime'],unit='ms')
        hist_data.sort_values(by=['datetime'],ascending=False)
        hist_data=hist_data.sort_values(by=['datetime'],ascending=True).reset_index()
        hist_data['Date']=hist_data['datetime'].dt.date
        hist_data=hist_data.drop(['index','datetime'],axis=1)
        hist_data['ticker'] = ticker
        if row_count!='Blank':
            return hist_data.tail(row_count)
        else:
            return hist_data
    except:
        df = pd.DataFrame()
        print('running except clause')
        return df
    
def get_fundamental_from_td(ticker,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    projection = 'fundamental'

    ##Define Payload
    payload = {'apikey': key,
               'symbol' : ticker,
                'projection': projection,
                }
    
    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()
    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        df = pd.DataFrame(dictlist[0][1]).T.reset_index(drop=True).iloc[0]
        return df
    except:
        print(dictlist)
        df = pd.DataFrame()
        print(ticker + " not valid.")
        return df
    
def get_yahoo_history(share_name):
    print("Pulling history")

    if share_name[-1]=='2':
        my_share = share.Share(share_name[:-1])
    else:
        my_share = share.Share(share_name)
    symbol_data = None

    try:
        symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,
                                              30000,
                                              share.FREQUENCY_TYPE_DAY,
                                              1)
        df = pd.DataFrame(symbol_data)
        df['timestamp'] = df['timestamp'].astype(str)
        df['timestamp'] = df['timestamp'].map(lambda x: x[:-3])
        df['Date'] =df['timestamp'].astype('int')
        df['Date'] = pd.to_datetime(df['Date'],unit='s')
        df=df.sort_values(by=['Date'],ascending=True)
        df['Date']=df['Date'].dt.date
        df=df.drop(['timestamp'],axis=1)
        df['ticker']=str(share_name.upper())
    except YahooFinanceError as e:
        print(e.message)
        sys.exit(1)
    return df

In [367]:
#ticker=tickers['Symbol'][1]
ticker='^GSPC'
ticker

'^GSPC'

In [ ]:
tickers.iloc[[1]]

In [ ]:
pd.DataFrame(get_fundamental_from_td(ticker,key)).T

In [179]:
print(ticker)
#td_data=get_td_price_hist(ticker,1,key,43)
td_data=get_td_price_hist(ticker,1,key)
print(len(td_data))
td_data.tail()

AOS
253


,open,high,low,close,volume,Date,ticker
248,82.68,82.96,81.8100,82.31,744896,2021-11-18,AOS
249,82.52,83.04,81.6505,82.46,726617,2021-11-19,AOS
250,82.88,83.48,82.3900,82.80,673088,2021-11-22,AOS
251,82.99,83.39,81.8710,82.45,620493,2021-11-23,AOS
252,82.16,82.93,81.9300,82.39,846838,2021-11-24,AOS


In [425]:
yahoo_data=get_yahoo_history(ticker)
print(len(yahoo_data))
yahoo_data

Pulling history
14843


,open,high,low,close,volume,Date,ticker
0,0.000000,62.580002,61.720001,62.320000,3700000,1962-12-11,^GSPC
1,0.000000,63.160000,62.130001,62.630001,3760000,1962-12-12,^GSPC
2,0.000000,63.070000,62.090000,62.419998,3380000,1962-12-13,^GSPC
3,0.000000,62.830002,61.959999,62.570000,3280000,1962-12-14,^GSPC
4,0.000000,62.950001,62.139999,62.369999,3590000,1962-12-17,^GSPC
...,...,...,...,...,...,...,...
14838,4708.439941,4717.750000,4694.220215,4697.959961,3265600000,2021-11-19,^GSPC
14839,4712.000000,4743.830078,4682.169922,4682.939941,3206280000,2021-11-22,^GSPC
14840,4678.479980,4699.390137,4652.660156,4690.700195,3428780000,2021-11-23,^GSPC
14841,4675.779785,4702.870117,4659.890137,4701.459961,2464040000,2021-11-24,^GSPC


In [182]:
#pd.merge(yahoo_data,td_data,on=['Date'],how='outer',indicator=True)
pd.merge(yahoo_data.tail(10),td_data.tail(10),on=['Date'],how='outer',indicator=True).describe()

,open_x,high_x,low_x,close_x,volume_x,open_y,high_y,low_y,close_y,volume_y
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,81.832999,82.647000,81.284999,82.119999,737040.000000,81.833000,82.645700,81.285150,82.120000,737038.200000
std,1.390643,1.095700,1.187812,0.912007,114492.107443,1.390644,1.096741,1.187883,0.912006,114498.036192
min,78.870003,79.879997,78.589996,79.669998,542200.000000,78.870000,79.875000,78.590000,79.670000,542178.000000
25%,81.857498,82.420000,81.207499,82.144997,668075.000000,81.857500,82.417750,81.207625,82.145000,668087.000000
50%,82.340000,82.950001,81.809998,82.419998,735750.000000,82.340000,82.950000,81.810000,82.420000,735756.500000
75%,82.670000,83.302500,81.915001,82.504997,830825.000000,82.670000,83.302500,81.915250,82.505000,830842.500000
max,82.989998,83.599998,82.389999,82.800003,891600.000000,82.990000,83.595000,82.390000,82.800000,891616.000000


In [426]:
yahoo_data['month']=pd.to_datetime(yahoo_data.Date).dt.month
yahoo_data['year']=pd.to_datetime(yahoo_data.Date).dt.year
yahoo_data['quarter']=pd.to_datetime(yahoo_data.Date).dt.quarter

dff=pd.read_csv('DFF.csv')

unrate=pd.read_csv('UNRATE.csv')
unrate['month']=pd.to_datetime(unrate.DATE).dt.month
unrate['year']=pd.to_datetime(unrate.DATE).dt.year
unrate.drop('DATE',axis=1,inplace=True)

gdp=pd.read_csv('GDPC1.csv')
gdp['quarter']=pd.to_datetime(gdp.DATE).dt.quarter
gdp['year']=pd.to_datetime(gdp.DATE).dt.year
gdp.drop('DATE',axis=1,inplace=True)

yahoo_data=pd.merge(yahoo_data,dff,left_on=pd.to_datetime(yahoo_data.Date),right_on=pd.to_datetime(dff.DATE),how='inner')
yahoo_data=pd.merge(yahoo_data,unrate,on=['month','year'],how='inner')
#yahoo_data=pd.merge(yahoo_data,gdp,on=['quarter','year'],how='inner')
yahoo_data.drop(['key_0','DATE','month','quarter','year'],axis=1,inplace=True)
yahoo_data

,open,high,low,close,volume,Date,ticker,DFF,UNRATE
0,0.000000,62.580002,61.720001,62.320000,3700000,1962-12-11,^GSPC,2.88,5.5
1,0.000000,63.160000,62.130001,62.630001,3760000,1962-12-12,^GSPC,2.88,5.5
2,0.000000,63.070000,62.090000,62.419998,3380000,1962-12-13,^GSPC,2.88,5.5
3,0.000000,62.830002,61.959999,62.570000,3280000,1962-12-14,^GSPC,3.00,5.5
4,0.000000,62.950001,62.139999,62.369999,3590000,1962-12-17,^GSPC,3.00,5.5
...,...,...,...,...,...,...,...,...,...
14819,4553.689941,4572.620117,4537.359863,4566.479980,3250210000,2021-10-25,^GSPC,0.08,4.6
14820,4578.689941,4598.529785,4569.169922,4574.790039,2866500000,2021-10-26,^GSPC,0.08,4.6
14821,4580.220215,4584.569824,4551.660156,4551.680176,3259510000,2021-10-27,^GSPC,0.08,4.6
14822,4562.839844,4597.549805,4562.839844,4596.419922,3197560000,2021-10-28,^GSPC,0.08,4.6


In [427]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html
# https://stackoverflow.com/questions/61319814/moving-average-in-pandas-issue-with-first-and-last-rows

days_out=252

yahoo_data['volume_moving'] = yahoo_data['volume'].rolling(days_out).mean().shift(periods=1)
yahoo_data['volume_moving_std']=yahoo_data['volume'].rolling(days_out).std().shift(periods=1)
yahoo_data['close_moving'] = yahoo_data['close'].rolling(days_out).mean().shift(periods=1)
yahoo_data['close_moving_std']=yahoo_data['close'].rolling(days_out).std().shift(periods=1)

# https://stackoverflow.com/questions/42138357/pandas-rolling-slope-calculation

def calc_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0]
    return slope

yahoo_data['volume_slope'] = yahoo_data['volume'].rolling(days_out).apply(calc_slope).shift(periods=1)
yahoo_data['close_slope'] = yahoo_data['close'].rolling(days_out).apply(calc_slope).shift(periods=1)

yahoo_data['close_future'] = yahoo_data['close'].shift(periods=-days_out)

yahoo_data.drop(columns=['high','low','volume','ticker','Date','close'],inplace=True)
yahoo_data.dropna(inplace=True)
yahoo_data

,open,DFF,UNRATE,volume_moving,volume_moving_std,close_moving,close_moving_std,volume_slope,close_slope,close_future
252,0.000000,3.50,5.5,4.519881e+06,9.292337e+05,69.227460,3.254833,4.207596e+03,0.041764,83.660004
253,0.000000,3.50,5.5,4.521944e+06,9.279823e+05,69.273452,3.238684,4.101602e+03,0.041550,83.449997
254,0.000000,3.50,5.5,4.524048e+06,9.268485e+05,69.318809,3.225283,4.007497e+03,0.041370,83.220001
255,0.000000,3.50,5.5,4.527619e+06,9.241526e+05,69.365952,3.210835,3.876007e+03,0.041184,83.550003
256,0.000000,3.50,5.5,4.535000e+06,9.215726e+05,69.414246,3.199715,3.815290e+03,0.041045,83.900002
...,...,...,...,...,...,...,...,...,...,...
14567,3464.899902,0.09,6.9,4.712277e+09,1.420226e+09,3129.904879,258.544825,2.306975e+06,1.177150,4566.479980
14568,3441.419922,0.09,6.9,4.712094e+09,1.420360e+09,3131.710831,259.294343,2.109958e+06,1.197375,4574.790039
14569,3403.149902,0.09,6.9,4.714545e+09,1.418567e+09,3133.212497,259.755008,1.914548e+06,1.212360,4551.680176
14570,3342.479980,0.09,6.9,4.716235e+09,1.417393e+09,3134.606386,260.191789,1.729118e+06,1.227693,4596.419922


In [428]:
yahoo_data.tail(50)

,open,DFF,UNRATE,volume_moving,volume_moving_std,close_moving,close_moving_std,volume_slope,close_slope,close_future
14522,3360.479980,0.09,8.4,4.589400e+09,1.492011e+09,3049.384638,225.648764,9.622158e+06,0.385578,4441.669922
14523,3386.010010,0.09,8.4,4.591906e+09,1.489878e+09,3051.214321,226.499267,9.383372e+06,0.405357,4479.529785
14524,3418.090088,0.09,8.4,4.595139e+09,1.487054e+09,3053.096106,227.397503,9.138589e+06,0.425746,4486.229980
14525,3435.949951,0.09,8.4,4.594311e+09,1.487480e+09,3055.414241,228.265191,8.994647e+06,0.441797,4496.189941
14526,3449.969971,0.09,8.4,4.597334e+09,1.484708e+09,3057.657258,229.293468,8.738148e+06,0.461538,4470.000000
14527,3485.139893,0.08,8.4,4.598209e+09,1.484143e+09,3060.076186,230.509099,8.557915e+06,0.483284,4509.370117
14528,3494.689941,0.09,8.4,4.601512e+09,1.481714e+09,3062.443687,231.794320,8.352626e+06,0.506902,4528.790039
14529,3509.729980,0.09,8.4,4.604209e+09,1.479726e+09,3064.758885,233.320501,8.147252e+06,0.535756,4522.680176
14530,3507.439941,0.09,7.8,4.609502e+09,1.476377e+09,3067.036068,234.761250,7.971231e+06,0.563626,4524.089844
14531,3543.760010,0.09,7.8,4.612106e+09,1.474861e+09,3069.497893,236.316318,7.809503e+06,0.591630,4536.950195


## Machine Learning

![FunnyReg](https://memegenerator.net/img/instances/49880835.jpg)

In [469]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor

def data_split(df,y_var,scale=False):
    reg_df=df.copy()

    # train test split
    #y=reg_df.pop(y_var)
    #X=reg_df
    #x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)
    
    # first 80% train
    x_train = reg_df.head(int(len(reg_df)*(0.8)))
    x_test = reg_df.iloc[max(x_train.index):]
    y_train = x_train.pop(y_var)
    y_test = x_test.pop(y_var)
    
    colz=x_train.columns
    if scale:
        scaler = StandardScaler()
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
        scaler.fit(y_train)
        y_train = scaler.transform(y_train)
        y_test = scaler.transform(y_test)
        
    return x_train, x_test, y_train, y_test, colz

def regression(x_train, x_test, y_train, y_test, colz):

    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(x_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(x_test)

    print("Number of training records:", len(y_train))
    print("Number of testing records:",len(y_test))
    print("\nLinear Regression Results")

    # The coefficients
    print('\nCoefficients:')
    for x,y in zip(colz,regr.coef_):
        print(x,y)
    
    #The intercept
    print('\nIntercept:', regr.intercept_)      
    print('\nLinear Regression R^2 score on training data: %.4f' % regr.score(x_train,y_train))
    print('Linear Regression R^2 score on test data: %.4f' % r2_score(y_test, y_pred))
    
def random_forest(x_train, x_test, y_train, y_test, colz):
    # If categorical y variable
    #random_forest = RandomForestClassifier(n_estimators=20)
    
    # If continous y variable
    random_forest = RandomForestRegressor(n_estimators=10)
    
    random_forest.fit(x_train, y_train)
    train_acc = random_forest.score(x_train, y_train)
    test_acc = random_forest.score(x_test, y_test)
    
    y_pred = random_forest.predict(x_test)
    
    print('Random Forest Results:')
    
    print('Training acuracy= ',train_acc)
    print('Test accuracy= ',test_acc)

    features = x_train.columns
    importances = random_forest.feature_importances_
    indices = np.argsort(importances)

    plt.subplots(figsize=(15, 11))
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

def cart(x_train, x_test, y_train, y_test, colz):
    cart = DecisionTreeRegressor(random_state=12)
    cart.fit(x_train, y_train)
    train_acc = cart.score(x_train, y_train)
    test_acc = cart.score(x_test, y_test)
    
    y_pred = cart.predict(x_test)
    
    print('CART Results:')
    
    print('CART training acuracy= ',train_acc)
    print('CART test accuracy= ',test_acc)
    
def ridge(x_train, x_test, y_train, y_test, colz):
    ridge = linear_model.Lasso(alpha=0.25)
    ridge.fit(x_train, y_train)
    y_pred = ridge.predict(x_test)
    train_acc = ridge.score(x_train, y_train)
    test_acc = ridge.score(x_test, y_test)
    
    print('Ridge Results:')
    print('Training acuracy =',train_acc)
    print('Test accuracy =',test_acc)

In [449]:
x_train, x_test, y_train, y_test, colz = data_split(yahoo_data,'close_future')

In [462]:
regression(x_train, x_test, y_train, y_test, colz)

Number of training records: 11456
Number of testing records: 2613

Linear Regression Results

Coefficients:
open 0.7957592354755617
DFF -3.7215137380683685
UNRATE -1.7772491582001144
volume_moving 2.0316249168631065e-07
volume_moving_std -1.016375497445425e-06
close_moving -0.13997376132967784
close_moving_std 3.6227629947461195
volume_slope 2.0429097007100053e-05
close_slope 62.65941247980581

Intercept: 134.5533168017326

Linear Regression R^2 score on training data: 0.9476
Linear Regression R^2 score on test data: 0.4447


In [470]:
ridge(x_train, x_test, y_train, y_test, colz)

Ridge Results:
Training acuracy = 0.9475587347419381
Test accuracy = 0.43705802991647025


## -------------PRACTICE-------------
1.

<a id='55'></a>
# Weekly Readings/Videos

https://blog.trinket.io/why-python/
    
https://towardsdatascience.com/top-16-python-applications-in-real-world-a0404111ac23

<a id='56'></a>
# Extra Practice